# CFD Domain's Creation & Discretization

**The next step for developing the CFD simulation for a case study, after the review of physics and theory behind the problem of interest, is to create the CFD/Computational domain. This CFD domain will first be created and then discretized into small finite elements (2D) or finite volumes (3D). The flow field's governing equations are then solved numerically inside these elements or volumes via an iterative process till a converged solution inside the CFD domain is obtained. This section explains the techniques and process of creation and discretization of the CFD domain for the problem of interest. It should be highlighted that this sections provides the most generic steps and techniques to create the CFD domain. Users can implement these steps/techniques in the mesher of their choice or they can use the following provided steps to produce the mesh in OpenFOAM.**

The domain consists of a rotating cylinder with diameter of 1 [m] surrounded by a larger cylinder with 64 [m] diameter. The inner cylinder would interact with the incoming flow while rotating around its central axis. The outer cylinder forms the exterior boundary of the domain. The dimension for the diameter of the inner cylinder is chosen to be unit in order to ease the calculation of various field variables such as Reynolds number. The diameter of the bounded cylinder is chosen to be 64 times the diameter of the main cylinder to ensure that the exterior boundaries do not effect the fluid flow in the near field of the rotating cylinder. A schematic of the geometry is shown in Fig.1:

<img src="./Images/1schematic.png" width="300">
Fig. 1. The schematic geometry of the domain.

The flow enters the CFD domain from the left half of the outer circle with a fixed velocity value and exits the domain from the right half with a specific pressure.

The meshing process is going to be done by blockMesh utility, the OpenFOAM native hexahedral mesher. Using blockMesh, one can divide the domain into multiple blocks, define mesh resolutions for each block and finally produce a high quality structural mesh. Here at first, two quadrant-shaped blocks which form the half geometry of the problem are going to be produced. Then by a mirroring process, the full geometry will be obtained. The two blocks are shown in Fig.2:

<img src="./Images/2blocks.png" width="500">
Fig. 2. The forming blocks of the geometry.

The meshing procedure using blockMesh is quite easy. At the first step the vertices of the blocks are going to be defined. Next the two mentioned blocks are going to be defined based on the previously mentioned vertices and the resolution of the edges are going to be specified. the last step would be defining the boundaries of the domain.

The blockMesh utility options are stored in a file named blockMeshDict which has to be located in the system directory of the working case. The blockMeshDict file can be copied from the tutorials provided by OpenFOAM itself. Among all tutorials, the cavity tutorial has the most basic written blockMeshDict file with only one block defined. The blockMeshDict file for this tutorial is located in $FOAM_Tutorials/incompressible/icoFoam/cavity/system. This file has to be copied into the system directory of our own case.

At the first line of the blockMeshDict file (After the header section of the file), There is a keyword named convertToMeters which does a scaling process on the final produced mesh. If the units are considered as [m] in a case (Which are in our case) then the value of this keyword would be 1.

<pre>convertToMeters 1;</pre>

The vertices of the blocks alongside with their labels are shown in Fig.3:

<img src="./Images/3labels.png" width="500">
Fig. 3. Labels of the vertices forming the blocks of the domain.

These vertices are defined in a dictionary of the same name in the blockMeshDict file:

<pre>
vertices
(
    (-0.5 0 0)
    (0 0.5 0)
    (0 32 0)
    (-32 0 0)
    (0.5 0 0)
    (32 0 0)

    (-0.5 0 1)
    (0 0.5 1)
    (0 32 1)
    (-32 0 1)
    (0.5 0 1)
    (32 0 1)
);
</pre>

As you may have mentioned, vertices are defined on two parallel planes along the z direction. In OpenFOAM there are no surface-type meshes. All meshes are in 3D, instead in 2D problems, the boundaries in the normal direction are defined as empty type. You can find better explanation on this type of boundary condition in [here](http://cfd.direct/openfoam/user-guide/boundaries/).

The vertices are automatically labeled by blockMesh by their order. The first vertex would be labeled as 0, the second vertex would be 1 and this numbering process continues up to the last vertex. The next step would be defining the two blocks based on the vertices. This is done by calling the vertices by their respective labels.

<pre>blocks
(
    hex (0 1 2 3 6 7 8 9) (25 200 1) simpleGrading (1 80 1)
    hex (1 4 5 2 7 10 11 8) (25 200 1) simpleGrading (1 80 1)
);</pre>

The block definitions are contained in a list named blocks. Each block definition is a compound entry consisting of a list of vertex labels whose order is described in the previous section, a vector giving the number of cells required in each direction, the type and list of cell expansion ratio in each direction. Here the number of cells is chosen to be 100 and 200 in angular and radial direction respectively. Also the total expansion ratio in the radial direction is chosen to be 80.

The next section defines the curved edges used in the blocks. Here there are four curved edges (quarter of a circle) for each block. The arc edges are defined by two end points and an axillary point in the middle of the edge.

<pre>edges
(
    arc 0 1 (-0.3535 0.3535 0)
    arc 2 3 (-22.6274 22.6274 0)
    arc 6 7 (-0.3535 0.3535 1)
    arc 8 9 (-22.6274 22.6274 1)

    arc 1 4 (0.3535 0.3535 0)
    arc 2 5 (22.6274 22.6274 0)
    arc 7 10 (0.3535 0.3535 1)
    arc 8 11 (22.6274 22.6274 1)
);</pre>

Finally its time to define the boundaries of the domain. The domain has five boundaries including: inlet, outlet, cylinder, symm and frontAndBack. Boundaries are face type geometries which are defined by the same vertices used in the blocking section. Each face is constructed by four vertices. A boundary can have one or more faces. In this section the basic boundary condition of each boundary is also defined. The vertices of a face are ordered as such that the normal vector of the face points to the inside of the domain.

<pre>boundary
(
    cylinder
    {
        type wall;
        faces
        (
            (1 0 6 7)
            (4 1 7 10)
        );
    }

    inlet
    {
        type patch;
        faces
        (
            (2 3 9 8)
        );
    }

    outlet
    {
        type patch;
        faces
        (
            (2 5 11 8)
        );
    }

    symm
    {
        type wall;
        faces
        (
            (0 3 9 6)
            (5 4 10 11)
        );
    }

    frontAndBack
    {
        type empty;
        faces
        (
            (1 2 3 0)
            (1 4 5 2)
            (6 9 8 7)
            (7 8 11 10)
        );
    }

);</pre>

The blockMeshDict file is now set up. Save the file and you can now make your half-geometry mesh by running the command:

<pre>blockMesh</pre>

The resulting mesh looks like Fig.4:

<img src="./Images/4halfMesh.png" width="500">
Fig. 4. The half-geometry mesh produced by blockMesh utility.

As mentioned before, the next step would be a mirroring process which gives us the full geometry of the domain. This is done by mirrorMesh utility. This utility, reflects the existing mesh with respect to the defined plane in the mirrorMeshDict file. By doing this the reflected boundaries are merged together. As always, the utility dictionary file is put in the system directory. The reflecting plane is defined as follows:

<pre>planeType       pointAndNormal;

pointAndNormalDict
{
    basePoint       (0 0 0);
    normalVector    (0 -1 0);
}

planeTolerance  1e-06;</pre>

The final step would be running the mirrorMesh command:

<pre>mirrorMesh -overwrite</pre>

The overwrite options allows OpenFOAM to replace the previously saved mesh by the new one. Now the final mesh is ready to use.

<img src="./Images/5finalMesh.png" width="500">
Fig. 5. The final mesh after using the mirrorMesh utility.